# Database Fundamentals and Applications &ndash; Group Assignment 3

In this assignment you will use SQL code to query `Yelp` from Python. You will use the SQL results to carry out several statistical analyses. You will be asked to discuss the results&mdash;**keep your answers brief; there is no need to write more than 2&ndash;4 sentences when interpreting results**.

Documentation of the dataset can be found in the main PDF (`E_EOR2_DBFA.GA3.pdf`) with instructions, found on Canvas.

## Getting started

### Connect Python to MySQL

Before Python can connect to your MySQL Server, you need to install the MySQL-Python connector. Run the code below to install this connector.

In [ ]:
# install mysql connector
!pip install mysql-connector-python

Please run the cell below to load all the necessary packages for this assignment. 

In [ ]:
# import packages
import os
import mysql.connector
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sma
%matplotlib inline

Use the code below to make a connection to the database on your local MySQL Server. **Do not forget to enter your password in the code below!**

In [ ]:
# connect to Yelp
connection = mysql.connector.connect(host = "localhost",
                                     user = "root",
                                     password = "your_password_goes_here", # do not forget to enter your password!
                                     db = "Yelp")

**If you run into errors executing the code above, try restaring the kernel**: in the 'Kernel' menu, click 'Restart'. If this does not help, try installing the newest MySQL-Python connector from https://dev.mysql.com/downloads/connector/python/8.0.html. Do not forget to restart the kernel after installation.

### Using Pandas' read_sql_query() to extract data from MySQL

In this assignment, we recommend you mix SQL statements with the functionalities of Python's Pandas package as well as a few other packages to extract, clean, and reshape data. The code below demostrates how to execute a SQL statement using Pandas, and extract a DataFrame.

In [ ]:
# select first 10k rows from Businesses
data = pd.read_sql_query("SELECT * \
                          FROM Businesses \
                          LIMIT 10000",
                         connection)
# let's inspect the data
data.head()

If everything works as it is supposed to, you should now see a table with 11 columns. Now you are ready to start the actual work on Group Assignment 3!

## Part 1

You have been asked to provide an overview of the reviews left by Yelp users. In particular, we are interested to investigate:

1.1 the number of reviews left in January, February, etc., aggregated across all years;

1.2 the evolution over time, of the number of reviews left per month;

1.3 difference in average star rating of reviews left in the morning, afternoon, evening, and at night; and

1.4 the relationship between number of reviews left by a user and how many stars they tend to give.

### 1.1 Number of reviews left in January, February, etc. (1 point)

- SQL: Write a SELECT statement that counts the number of reviews left in each month, aggregated across all years. So how many reviews have been left in January across all years, in February across all years, etc. Use SQL's `MONTH()` function to convert the timestamp in attribute `date` to a number beween 1 and 12, where 1 corresponds to January and 12 to December, and use this number to group by, and to order by.
- Python: Execute the SQL query. Visualise the data in the resulting DataFrame using a bar plot.
- Comment on the results: In which month are most reviews left? And in which month the fewest?

In [ ]:
# extract required data


In [ ]:
# visualise the results with a barplot, with stars (x-axis) and counts (y-axis)
sns.barplot(data = ...,
           x = "...",
           y = "...")

**Your discussion:** 

### 1.2 Evolution of reviews per month over time (1 point)

- SQL: Write a SELECT statement that counts the number of reviews left in each month of each year. Use SQL's `MONTH()` and `YEAR()` functions to convert the timestamp in attribute `date` to a month and year attribute (both as a number). Group and order by these two attributes. Think carefully about the order of appearance: should you first order by year and then by month? Or the other way around?
- Python: Execute the SQL query. Visualise the data in the resulting DataFrame using a line plot.
- Comment on the results: How are the number of monthly reviews developing over time? And do you see the same seasonalities as in the previous question?

In [ ]:
# extract required data


In [ ]:
# visualise the results with a line plot, with stars (x-axis) and counts (y-axis)
plt.plot(data["..."])
plt.title("...")
plt.xlabel("Time $t$ in months, starting from October 2004 ($t=0$)")
plt.ylabel("...")
plt.show()

**Your discussion:** 

### 1.3 Average star rating of reviews left in the morning, afternoon, evening, and at night (2 points)

- SQL: Write a SELECT statement that returns for each review the number of `stars` as well as another attribute called `daypart` which equals 'Night' in case the review was left before 06:00, 'Morning' if left before '12:00', 'Afternoon' if left before '18:00', and 'Evening' otherwise. You may find a `CASE WHEN END` statement and `HOUR()` useful to carry out this task.
- Python: Execute the SQL query. Transform the `daypart` column into a set of four dummy variables. Choose 'Morning' as base category. Regress `stars` on an intercept and the dummies for 'Afternoon', 'Evening', and 'Night'. Report the coefficient and their *t*-test statistics.
- Comment on the results: Are the differences across parts of the day different? When are the most favourable reviews left? And when the least favourable?

In [ ]:
# extract the required data


In [ ]:
# create dummies from the daypart attribute
dummies = pd.get_dummies(data["daypart"])
# regress stars on 3 out of 4 dummies and the intercept
model = sma.OLS(data["..."], sma.add_constant(dummies[["Afternoon","...","..."]])).fit()
# report the regression summary
model.summary()

**Your discussion:** 

### 1.4 Relationship between number of reviews and stars (1 point)

- SQL: Write a SELECT statement that, for each review, returns `stars`, as well as the total number of reviews written by the user who left that particular review. Hint: use a window function. Exclude users that joined less than a year before the user that was latest to join as a reviewer. Users that have never left a review should also be ignored. 
- Python: Execute the SQL query. **Warning!** Due to the size of data, this query can easily take up to 15 minutes to run. Use a pie chart to visualise the star distribution among users that have left only 1 review. Create the same type of pie chart for users that have left 10 or more reviews.
- Comment on the result: What main differences do you see between the two pie charts?

In [ ]:
# extract the required data


In [ ]:
# create the two pie charts
data_1_review = data[data.no_reviews == ...]
data_1_grouped = data_1_review.groupby("stars").count().reset_index()
data_10_reviews = data[data.no_reviews >= ...]
data_10_grouped = data_10_reviews.groupby("stars").count().reset_index()

plt.pie(x = data_1_grouped.no_reviews, labels = data_1_grouped.stars)
plt.title("1 review")
plt.show()

...

**Your discussion:** 

## Part 2

You are now asked to provide an overview of businesses and their categories. In particular, we are interested to see
- the most common business categories,
- the distribution of average star rating per category, and
- if there are any significant differences between categories in terms of star rating.

### 2.1 Top categories (2 points)

- SQL: Write a SELECT statement that returns the categories column from the Businesses table. To cut down on the amount of data, randomly sample around 5% of the data in this table. For this sampling, you may find this useful: https://phpfog.com/select-random-sample-records-mysql/. Make sure to remove rows with missing categories values.
- Python: Execute the SQL query. The categories column has a list of categories that each business falls under. Extract every category name from each row list and store it in one big list. Display the 5 most common categories and their counts. Here, you might find these links useful: https://www.w3schools.com/python/ref_string_split.asp; https://docs.python.org/3/library/collections.html#collections.Counter.most_common.

In [ ]:
# extract the required data


In [ ]:
# extract separate categories from categories list column


In [ ]:
# give counts of the 5 most common categories


### 2.2 Review differences between top categories (2 points)

- SQL: Write a SELECT statement that returns an average rating of each business. Create an extra column that returns only one category based on what is contained in the categories column list. The five categories that interest us are (sorted from most to least important): (1) 'Restaurants', (2) 'Shopping', (3) 'Food', (4) 'Home Services', (5) 'Beauty & Spas'. E.g. if the list of the categories column includes the word 'Restaurants' your column should return only 'Restaurants' and ignore any other categories in the list. If, for example, the categories list contains ['Shopping', 'Food', 'Restaurants'] your column should only return 'Restaurants'. You may find the LIKE and CASE WHEN statements useful to carry out the task. Businesses that have less than 5 reviews or do not fall into any category we are interested in should be ignored.
- Python: Execute the SQL query. **Warning!** Due to the size of data, this query can easily take up to 15 minutes to run. Visualise the distribution of average star reviews for each category in seperate plots. Describe what you observe.

In [ ]:
# extract the required data


In [ ]:
# visualise the distribution of average star rating with 5 separate histograms (for each category)


**Your discussion:**

### This brings us to the end of Group Assignment 3. Do not forget to save your Jupyter Notebook!